In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt


In [ ]:
from tensorflow.keras.datasets import mnist

# Load dataset
(X_train, y_train), (_, _) = mnist.load_data()

# Normalize images to [-1, 1] (as tanh activation in generator outputs values in this range)
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = np.expand_dims(X_train, axis=-1)

# One-hot encode the labels
num_classes = 10
y_train_one_hot = tf.keras.utils.to_categorical(y_train, num_classes)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
def build_generator(latent_dim, num_classes):
    # Label embedding and noise input
    label_input = layers.Input(shape=(num_classes,))
    noise_input = layers.Input(shape=(latent_dim,))

    # Concatenate label and noise
    merged_input = layers.Concatenate()([noise_input, label_input])

    # Build the generator network
    x = layers.Dense(256, activation="relu")(merged_input)
    x = layers.Dense(512, activation="relu")(x)
    x = layers.Dense(1024, activation="relu")(x)
    x = layers.Dense(28 * 28 * 1, activation="tanh")(x)
    output = layers.Reshape((28, 28, 1))(x)

    return tf.keras.Model([noise_input, label_input], output)


In [ ]:
def build_discriminator(input_shape, num_classes):
    # Image input
    image_input = layers.Input(shape=input_shape)

    # Label input
    label_input = layers.Input(shape=(num_classes,))
    label_embedding = layers.Dense(np.prod(input_shape))(label_input)
    label_embedding = layers.Reshape(input_shape)(label_embedding)

    # Concatenate image and label
    merged_input = layers.Concatenate()([image_input, label_embedding])

    # Build the discriminator network
    x = layers.Flatten()(merged_input)
    x = layers.Dense(1024, activation="relu")(x)
    x = layers.Dense(512, activation="relu")(x)
    x = layers.Dense(256, activation="relu")(x)
    output = layers.Dense(1, activation="sigmoid")(x)

    return tf.keras.Model([image_input, label_input], output)


In [ ]:
def build_cgan(generator, discriminator):
    discriminator.trainable = False

    # Inputs for the generator
    noise_input = layers.Input(shape=(latent_dim,))
    label_input = layers.Input(shape=(num_classes,))

    # Generate fake image
    fake_image = generator([noise_input, label_input])

    # Discriminator's prediction for the fake image
    validity = discriminator([fake_image, label_input])

    return tf.keras.Model([noise_input, label_input], validity)


In [ ]:
latent_dim = 100
input_shape = (28, 28, 1)

# Build and compile the generator and discriminator
generator = build_generator(latent_dim, num_classes)
discriminator = build_discriminator(input_shape, num_classes)
discriminator.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Build and compile the cGAN
cgan = build_cgan(generator, discriminator)
cgan.compile(loss="binary_crossentropy", optimizer="adam")

# Training parameters
epochs = 10000
batch_size = 64

# Labels for real and fake images
real = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

# Training loop
for epoch in range(epochs):
    # ---------------------
    # Train Discriminator
    # ---------------------
    # Select a random batch of real images and labels
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    real_images, real_labels = X_train[idx], y_train_one_hot[idx]

    # Generate a batch of fake images
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    fake_labels = np.random.randint(0, num_classes, batch_size)
    fake_labels_one_hot = tf.keras.utils.to_categorical(fake_labels, num_classes)
    fake_images = generator.predict([noise, fake_labels_one_hot])

    # Train the discriminator
    d_loss_real = discriminator.train_on_batch([real_images, real_labels], real)
    d_loss_fake = discriminator.train_on_batch([fake_images, fake_labels_one_hot], fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # ---------------------
    # Train Generator
    # ---------------------
    # Generate noise and fake labels
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    random_labels = np.random.randint(0, num_classes, batch_size)
    random_labels_one_hot = tf.keras.utils.to_categorical(random_labels, num_classes)

    # Train the generator (via the combined model)
    g_loss = cgan.train_on_batch([noise, random_labels_one_hot], real)

    # Print the progress
    if epoch % 100 == 0:
        print(f"{epoch} [D loss: {d_loss[0]:.4f}, acc.: {100 * d_loss[1]:.2f}%] [G loss: {g_loss:.4f}]")

    # Save generated images at intervals
    if epoch % 1000 == 0:
        noise = np.random.normal(0, 1, (10, latent_dim))
        sample_labels = np.eye(num_classes)
        gen_images = generator.predict([noise, sample_labels])

        # Rescale images to [0, 1] for visualization
        gen_images = 0.5 * gen_images + 0.5

        # Plot the generated images
        fig, axs = plt.subplots(1, 10, figsize=(20, 2))
        for i in range(10):
            axs[i].imshow(gen_images[i, :, :, 0], cmap="gray")
            axs[i].axis("off")
        plt.show()


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step 


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


TypeError: unsupported format string passed to list.__format__

In [ ]:
# Training the cGAN
for epoch in range(epochs):
    # ---------------------
    # Train Discriminator
    # ---------------------
    # Select a random batch of real images and corresponding labels
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    real_images, real_labels = X_train[idx], y_train_one_hot[idx]

    # Generate a batch of fake images
    noise = np.random.normal(0, 1, (batch_size, latent_dim))  # Random noise
    fake_labels = np.random.randint(0, num_classes, batch_size)  # Random labels
    fake_labels_one_hot = tf.keras.utils.to_categorical(fake_labels, num_classes)  # One-hot encode labels
    fake_images = generator.predict([noise, fake_labels_one_hot])  # Generate fake images

    # Targets for real and fake images
    real_targets = np.ones((batch_size, 1))  # Real images target = 1
    fake_targets = np.zeros((batch_size, 1))  # Fake images target = 0

    # Train the discriminator
    d_loss_real = discriminator.train_on_batch([real_images, real_labels], real_targets)
    d_loss_fake = discriminator.train_on_batch([fake_images, fake_labels_one_hot], fake_targets)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # ---------------------
    # Train Generator
    # ---------------------
    # Generate noise and fake labels for the generator
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    random_labels = np.random.randint(0, num_classes, batch_size)
    random_labels_one_hot = tf.keras.utils.to_categorical(random_labels, num_classes)

    # Train the generator (via the combined model)
    g_loss = cgan.train_on_batch([noise, random_labels_one_hot], real_targets)

   # Training the cGAN
for epoch in range(epochs):
    # ... (rest of the code)

    # Train the discriminator
    d_loss_real = discriminator.train_on_batch([real_images, real_labels], real_targets)
    d_loss_fake = discriminator.train_on_batch([fake_images, fake_labels_one_hot], fake_targets)
    d_loss = 0.5 * np.add(d_loss_real[0], d_loss_fake[0])  # Extract the loss values
    d_acc = 0.5 * np.add(d_loss_real[1], d_loss_fake[1]) # Extract the accuracy values

    # ... (rest

    # Save generated images at every 1000 epochs
    if epoch % 1000 == 0:
        noise = np.random.normal(0, 1, (10, latent_dim))  # Generate noise
        sample_labels = np.eye(num_classes)  # One-hot labels for digits 0-9
        gen_images = generator.predict([noise, sample_labels])  # Generate fake images

        # Rescale images to [0, 1] for visualization
        gen_images = 0.5 * gen_images + 0.5

        # Plot the generated images
        fig, axs = plt.subplots(1, 10, figsize=(20, 2))
        for i in range(10):
            axs[i].imshow(gen_images[i, :, :, 0], cmap="gray")  # Display the image
            axs[i].set_title(f"Class {i}")
            axs[i].axis("off")
        plt.show()


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
2/2 ━━━━━━━━